# --- Day 17: Trick Shot --- 

https://adventofcode.com/2021/day/17

## Get Input Data

In [1]:
def parse_data(str):
    """Parse data for Trick Shot."""

    import re

    range_values = re.findall('-?\d+', str) 

    target_range = {
        'x_min' : int(range_values[0]),
        'x_max' : int(range_values[1]),
        'y_min' : int(range_values[2]),
        'y_max' : int(range_values[3])
    }

    return target_range

In [2]:
test_target = parse_data('target area: x=20..30, y=-10..-5')
test_target

{'x_min': 20, 'x_max': 30, 'y_min': -10, 'y_max': -5}

In [3]:
target = parse_data('target area: x=288..330, y=-96..-50')
target

{'x_min': 288, 'x_max': 330, 'y_min': -96, 'y_max': -50}

## Part 1
---

In [4]:
def in_target(position, target):
    """Determine whether a given x, y position is within the target range."""

    if ((target['x_min'] <= position['x'] <= target['x_max']) and 
        (target['y_min'] <= position['y'] <= target['y_max'])):
        return True

    else:
        return False

In [5]:
position = {'x':25, 'y':-8}
in_target(position, test_target)  # Should return True

True

In [6]:
position = {'x':0, 'y':10}
in_target(position, test_target)  # Should return False

False

In [7]:
def next_step(position, velocity):
    """Return the next x, y position and x, y velocity."""

    position['x'] += velocity['x']
    position['y'] += velocity['y']

    if velocity['x'] > 0:
        velocity['x'] -= 1
    elif velocity['x'] < 0:
        velocity['x'] += 1

    velocity['y'] -= 1

    return position, velocity

In [8]:
position = {'x':0, 'y':0}; velocity = {'x':10, 'y':10}
next_step(position, velocity)  # Should return position{'x':10, 'y':10}, velocity{'x':9, 'y':9}

({'x': 10, 'y': 10}, {'x': 9, 'y': 9})

In [9]:
def run_steps(position, velocity, target):
    """Run through steps to see if a given x, y velocity lands in the given target.
    (All starting positions are at 0, 0.)

    If it does hit the target, return the maximum height of the arc.
    """
    
    position = position.copy()
    velocity = velocity.copy()

    max_y = position['y']

    while position['y'] >= target['y_min'] and position['x'] <= target['x_max']:
        max_y = max(max_y, position['y'])
        
        if in_target(position, target):
            return max_y
        else:
            position, velocity = next_step(position, velocity)
        # print(position)

    return 'missed target'

In [10]:
position = {'x':0, 'y':0}; velocity = {'x':7, 'y':2}
run_steps(position, velocity, test_target)  # Should return 3

3

In [11]:
position = {'x':0, 'y':0}; velocity = {'x':6, 'y':3}
run_steps(position, velocity, test_target)  # Should return 6

6

In [12]:
position = {'x':0, 'y':0}; velocity = {'x':9, 'y':0}
run_steps(position, velocity, test_target)  # Should return 0

0

In [13]:
position = {'x':0, 'y':0}; velocity = {'x':17, 'y':-4}
run_steps(position, velocity, test_target)  # Should return 'missed target'

'missed target'

In [14]:
position = {'x':0, 'y':0}; velocity = {'x':6, 'y':9}
run_steps(position, velocity, test_target)  # Should return 45

45

In [15]:
position = {'x':0, 'y':0}; velocity = {'x':6, 'y':10}
run_steps(position, velocity, test_target)  # Should return 'missed target'

'missed target'

In [16]:
position = {'x':0, 'y':0}; velocity = {'x':5, 'y':3}
run_steps(position, velocity, test_target)  # Should return 'missed target' -- undershoots target

'missed target'

In [17]:
def find_x_velocity(target):
    """Find the x velocity that produces a vertical drop over the target.
    
    The peak x velocity will be one that is a triangle number within the target range.
    There could be more than one, so I will have to test each of them.
    """

    x_start = 0
    triangle_number = 0

    while triangle_number < target['x_min']:
        x_start += 1
        triangle_number = (x_start * (x_start + 1)) / 2

    x_stop = x_start
    while triangle_number < target['x_max']:
        x_stop += 1
        triangle_number = (x_stop * (x_stop + 1)) / 2

    return x_start, x_stop - 1

In [18]:
find_x_velocity(test_target)  # Should return 6, 7

(6, 7)

In [19]:
def find_y_velocity(target):
    """Find the y velocity that produces the peak value and still lands in the target.
    
    The peak y velocity will be when there's just a single step left to get from 0 to the 
    target. That is, the peak y velocity will produce a target hit on the bottom line of the
    target, in one of the x columns that are triangle numbers.
    """

    y_velocity = abs(target['y_min']) - 1
    return y_velocity

In [20]:
find_y_velocity(test_target)  # Should return 9

9

In [21]:
def get_answer(target):

    x_start, x_stop = find_x_velocity(target)
    y_velocity = find_y_velocity(target)

    position = {'x':0, 'y':0}
    max_ys = []

    for x_velocity in range(x_start, x_stop+1):
        velocity = {'x':x_velocity, 'y':y_velocity}
        max_y = run_steps(position, velocity, target)
        max_ys.append(max_y)

    return max(max_ys)

### Run on Test Data

In [22]:
get_answer(test_target)  # Should return 45

45

### Run on Input Data

In [23]:
get_answer(target)

4560

## Part 2
---

In [24]:
def hits_target(position, velocity, target):
    """Return initial velocity if it hits the target; return 'missed target' otherwise."""

    position = position.copy()
    initial_velocity = velocity.copy()
    velocity = velocity.copy()

    while position['y'] >= target['y_min'] and position['x'] <= target['x_max']:

        if in_target(position, target):
            return initial_velocity
        else:
            position, velocity = next_step(position, velocity)
        # print(position)

    return 'missed target'

In [25]:
def get_answer2(target):
    """Return a count of all the possible initial velocities that land in the target."""

    position = {'x':0, 'y':0}
    hits = []

    for x in range(target['x_max']+1):
        for y in range(target['y_min'], abs(target['y_min'])+1):

            velocity = {'x':x, 'y':y}
            hit = hits_target(position, velocity, target)

            if hit != 'missed target':
                hits.append(hit)

    return len(hits)

### Run on Test Data

In [26]:
get_answer2(test_target)  # Should return 112

112

### Run on Input Data

In [27]:
get_answer2(target)

3344